# Домашнее задание к занятию
## "Проблема качества данных"
Необходимо запустить практическую часть занятия, и посмотреть самому то, о чём говорили на лекции. По образу практики, попробуйте создать искусственный датасет с лишними столбцами. Целевую метку, при правильной обработке данных, формируйте таким образом, чтобы без затруднений её смогла описать линейная модель. Ориентируйтесь на то, что было показано во время занятия, и каждый шаг описывайте в markdown. Здесь важно видеть ваш ход мысли. Не бойтесь ошибиться или написать не то. Данное задание не имеет какого-то “правильного” решения. Цель - достичь базового понимания проблемы. Чем больше вы фантазируете, тем лучше :) Тем не менее, старайтесь представить те ситуации, которые по-вашему мнению могли бы быть в реальных данных. 

### Решение:

Практическая часть занятия была запущена в соответствующем ноутбуке, в результате чего на практике убедилась, что работа с признаками очень важна при построении моделей. Грамотная обработка признаков позволяет значительно улучшить качество модели.

Далее создаем искусственный датасет с лишними столбцами.

В создаваемом датасете содержится информация о студентах некого курса. Описание признаков приводится в коде при создании соответствующих значений. Целевая пременная - это информация о количестве набранных баллов студентом при написании финальной работы.

In [1]:
# Заружаем необходимые библиотеки
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [2]:
'''
Создаем датасет
Количество строк датасета
'''
n_samples = 1000 
'''
Средняя оценка студента по выполненным работам за время обучения на курсе.
Оценивается по десятибальной шкале. 
При получении оценки 1 и выше работа считается сданной
'''
average_rating = np.random.choice(10, n_samples) + 1 
'''
Количество сданных работ
'''
number_ratings = np.random.choice(11, n_samples) + 5
'''
Пол студента, где 1 - женский пол, а 0 - мужской
'''
gender = np.random.choice(2, n_samples)
'''
Возраст студента курса
'''
age = np.random.choice(50, n_samples) + 18
'''
Число посещений занятий в процентах
'''
number_visits = np.random.choice(61, n_samples) + 40
'''
Количество баллов при написании финальной работы. Максимальное заначение - 166.
По усмотрению преподавателя при неправильном ответе, но верных рассуждениях возможно ставить по половине балла, 
что сильно влияет на итоговую оценку при наличии многих "половинных" баллах 
и помогает улучшить оценку у невнимательных, но способных студентов. 
На итоговую оценку не влияет возраст, но немного влияет пол (личные предпочтения преподавателя).
Данный показатель нелинейно зависит от средней оценки и количества сданных работ  
'''
final_score = average_rating * number_ratings + 2 * gender + number_visits / 2 - 36 # 
'''
Объедняем данные в датасет
'''
data = pd.DataFrame({'average_rating': average_rating, 'number_ratings': number_ratings, 'gender': gender, 
                     'age': age, 'number_visits': number_visits, 'final_score': final_score})
data.head(5)

,average_rating,number_ratings,gender,age,number_visits,final_score
0,8,8,1,51,95,77.5
1,10,10,1,59,69,100.5
2,5,11,0,20,53,45.5
3,2,12,1,64,71,25.5
4,4,5,1,50,97,34.5


In [3]:
# Формируем модель машинного обучения для всех признаков
X = data[['average_rating', 'number_ratings', 'gender', 'age', 'number_visits']]
y = data['final_score']
reg = LinearRegression().fit(X, y)
# Выводим полученные коэффициенты
print('Weights: {}'.format(reg.coef_))
# Выводим полученное смещение
print('Bias: {}'.format(reg.intercept_))
# Строим предсказание
pred_values = reg.predict(X)
# Выводим среднюю абсолютную ошибку
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 9.98283975e+00  5.42050181e+00  2.09330401e+00 -3.04469720e-03
  5.13045732e-01]
Bias: -90.86293580104689
Error: 6.663099166734237


Из полученных данных видим, что в среднем наша модель ошибается на 7 баллов. Полученные коэффициенты у линейно зависмых параметров приблизительно совпадают (при большом допущении). Предсказнное смещение сильно отличается от заданного. Кроме того, у признака возраста студентов небольшой коэффициент, следовательно, данный признак не влияет или незначительно влияет на целевую переменную.

Попробуем улучшить модель.

In [4]:
# Формируем модель машинного обучения без учета признака 'age' (возраст студента)
# Предполагаем, что незначимость признака определили на основе анализа данных
X = data[['average_rating', 'number_ratings', 'gender', 'number_visits']]
y = data['final_score']
reg = LinearRegression().fit(X, y)
# Выводим полученные коэффициенты
print('Weights: {}'.format(reg.coef_))
# Выводим полученное смещение
print('Bias: {}'.format(reg.intercept_))
# Строим предсказание
pred_values = reg.predict(X)
# Выводим среднюю абсолютную ошибку
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [9.98236803 5.42104121 2.09272148 0.51306688]
Bias: -90.99566461660564
Error: 6.662746488904313


Полученные значения для модели без учета признака, не влияющего на целевую переменную, практически не изменились. Следовательно, учитывать "лишние" признаки при формировании модели не требуется. 

Следующим шагом улучшения модели будет создание новых признаков. Первый новый признак не будет связан с зависимостью целевой переменной, которую мы создали. Второй признак указан при создании целевой переменной. Таким образом, мы увидим зависимость качества модели не только при создании нужных дополнительных признаков, но и тех, которые фактически не обладают теми качествами, которые мы предполагаем.

In [5]:
# Создаем новые признаки
data['mult_1'] = data['number_visits'] * data['gender']
data['mult_2'] = data['average_rating'] * data['number_ratings']
data.head(5)

,average_rating,number_ratings,gender,age,number_visits,final_score,mult_1,mult_2
0,8,8,1,51,95,77.5,95,64
1,10,10,1,59,69,100.5,69,100
2,5,11,0,20,53,45.5,0,55
3,2,12,1,64,71,25.5,71,24
4,4,5,1,50,97,34.5,97,20


In [6]:
# Формируем модель машинного обучения без учета признака 'age' (возраст студента) и с первым созданным признаком
X = data[['average_rating', 'number_ratings', 'gender', 'number_visits', 'mult_1']]
y = data['final_score']
reg = LinearRegression().fit(X, y)
# Выводим полученные коэффициенты
print('Weights: {}'.format(reg.coef_))
# Выводим полученное смещение
print('Bias: {}'.format(reg.intercept_))
# Строим предсказание
pred_values = reg.predict(X)
# Выводим среднюю абсолютную ошибку
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [9.98012736 5.42402283 0.50673456 0.50140653 0.02280451]
Bias: -90.20527132247224
Error: 6.656732807397353


Полученные значения для модели с созданным признаком, который не описывает реальные закономерности, незначительно изменились. Следовательно, при создании новых признаков требуется понимать их действительное влияние на целевую переменную.

In [7]:
# Формируем модель машинного обучения без учета признака 'age' (возраст студента) и со вторым созданным признаком
X = data[['average_rating', 'number_ratings', 'gender', 'number_visits', 'mult_2']]
y = data['final_score']
reg = LinearRegression().fit(X, y)
# Выводим полученные коэффициенты
print('Weights: {}'.format(reg.coef_))
# Выводим полученное смещение
print('Bias: {}'.format(reg.intercept_))
# Строим предсказание
pred_values = reg.predict(X)
# Выводим среднюю абсолютную ошибку
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [-1.02842493e-15  3.10862447e-15  2.00000000e+00  5.00000000e-01
  1.00000000e+00]
Bias: -36.00000000000008
Error: 2.276223654007481e-14


Из полученных значений видим, что качество модели значительно улучшилось. Возникающая ошибка минимальна. Создание и использование нового признака, реально описывающего целевую переменную, позволило создать модель, которая достаточно точно уловила существующую закономерность. 

Обращаем внимание, что первые два признака обладают малыми коэффициентами, следовательно, ими можно пренебречь при формировании модели.

In [8]:
# Формируем итоговую модель машинного обучения на основании ранее указнных фактов
X = data[['gender', 'number_visits', 'mult_2']]
y = data['final_score']
reg = LinearRegression().fit(X, y)
# Выводим полученные коэффициенты
print('Weights: {}'.format(reg.coef_))
# Выводим полученное смещение
print('Bias: {}'.format(reg.intercept_))
# Строим предсказание
pred_values = reg.predict(X)
# Выводим среднюю абсолютную ошибку
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [2.  0.5 1. ]
Bias: -35.99999999999992
Error: 1.77315939708933e-14


Итоговая модель достаточно полно описывает целевую переменную на основании данных и отображает ту зависимоть целевой переменной, которую мы задали.